In [1]:
import requests
import pandas as pd
from yelp_secret_key import api_key, api_key_secret

In [2]:
import json
import time

In [7]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('pre_processed.csv')

In [8]:
# Drop rows with NaN values in categories column
df = df.dropna(subset=['categories'])

# Keep only rows where categories contain 'restaurants' or 'food'
df = df[df['categories'].str.contains('restaurants|food', case=False, regex=True)]

# Get the length of the resulting dataframe
length = len(df)

print(f"Length of dataset: {length}")

Length of dataset: 239065


# pull reviews

In [3]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': 'Bearer ' + api_key_secret,
}


In [13]:
df['postal_code'].unique()

array([93101., 93117., 93105., 93013., 93110., 93108., 93118., 93103.,
       93109., 93111., 93106., 93190., 93041., 93067., 93102., 95050.,
       93031., 96161., 93121., 93199.])

In [15]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    'Authorization': 'Bearer ' + api_key_secret,
}


# Set up API parameters
params = {
    'categories': 'restaurants',
    'limit': 50
}

# Define zipcodes to search
zipcodes = ['93101.', '93117.',' 93105.', '93013.', '93110.', '93108.', '93118.', '93103.',
       '93109.', '93111.', '93106.', '93190.', '93041.', '93067.', '93102.', '95050.',
       '93031.', '96161.', '93121.', '93199.']

# Count total number of restaurants
total_restaurants = 0

# Iterate over zipcodes and count restaurants in each
for zipcode in zipcodes:
    # Set location parameter to current zipcode
    params['location'] = zipcode
    # Make API call and parse response
    response = requests.get(url, headers=headers, params=params)
    data = json.loads(response.text)
    # Get total number of restaurants in current zipcode
    num_restaurants = data['total']
    # Add current count to total count
    total_restaurants += num_restaurants
    # Print current count
    print(f"{zipcode}: {num_restaurants}")
    
# Print total number of restaurants across all zipcodes
print(f"Total restaurants: {total_restaurants}")

93101.: 494
93117.: 56
 93105.: 191
93013.: 71
93110.: 664
93108.: 590
93118.: 259
93103.: 488
93109.: 498
93111.: 410
93106.: 167
93190.: 1500
93041.: 1100
93067.: 52
93102.: 38
95050.: 3200
93031.: 6
96161.: 98
93121.: 57
93199.: 39
Total restaurants: 9978


In [19]:
import pandas as pd
import requests
import json

# Set up the API endpoint and headers
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'

headers = {
    'Authorization': f'Bearer {api_key_secret}',
    'Content-Type': 'application/json'
}

# Define the parameters for the API request
zipcodes = ['93101.', '93117.',' 93105.', '93013.', '93110.', '93108.', '93118.', '93103.',
       '93109.', '93111.', '93106.', '93190.', '93041.', '93067.', '93102.', '95050.',
       '93031.', '96161.', '93121.', '93199.']  
df_restaurants = pd.DataFrame(columns=['id', 'name', 'category', 'rating', 'price', 'address', 'latitude', 'longitude'])
params = {
    'categories': 'restaurants',
    'limit': 50,
    'offset': 0
}

# Send a GET request to the API endpoint with the parameters and headers for each zip code
for zipcode in zipcodes:
    params['location'] = zipcode
    total_results = None
    while total_results is None or params['offset'] < total_results:
        response = requests.get(API_HOST + SEARCH_PATH, headers=headers, params=params)

        # Check if there was an error in the request
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            print(response.text)
            break

        # Parse the JSON response and extract the relevant information
        data = json.loads(response.text)
        for business in data['businesses']:
            categories = [category['title'] for category in business['categories']]
            df_restaurants = df_restaurants.append({
                'id': business['id'],
                'name': business['name'],
                'category': categories,
                'rating': business['rating'],
                'price': business.get('price', None),
                'address': ' '.join(business['location']['display_address']),
                'latitude': business['coordinates']['latitude'],
                'longitude': business['coordinates']['longitude']
            }, ignore_index=True)

        # Update the offset parameter to get the next page of results
        params['offset'] += params['limit']

        # Update the total results count
        if total_results is None:
            total_results = data['total']
    
    # Reset the offset parameter for the next zip code
    params['offset'] = 0

# Print the total number of restaurants found
print(f"Total restaurants: {len(df_restaurants)}")

# Save the dataframe to a CSV file
#df_restaurants.to_csv('restaurants.csv', index=False)


<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

Error: 400
{"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 1000."}}


<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

Error: 400
{"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 1000."}}


<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

Error: 400
{"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 1000."}}


<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

Total restaurants: 7178


<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_restaurants = df_restaurants.append({
<ipython-input-19-24ab5f08bf49>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [20]:
df_restaurants

,id,name,category,rating,price,address,latitude,longitude
0,6EizP0XyqITuGZ8GZMI9Lw,Toma Restaurant & Bar,"[Bars, Italian]",4.5,$$$,"324 West Cabrillo Blvd Santa Barbara, CA 93101",34.408633,-119.693496
1,tthy48ZIX_wfY8Dy0Dvg6w,Bouchon,"[French, American (New), Seafood]",4.5,$$$,"9 W Victoria St Santa Barbara, CA 93101",34.423983,-119.705860
2,yPSejq3_erxo9zdVYTBnZA,Los Agaves,"[Tacos, Bars]",4.5,$$,"600 N Milpas St Santa Barbara, CA 93101",34.427530,-119.686570
3,KSYONgGtrK0nKXfroB-bwg,Loquita,"[Spanish, Tapas Bars, Seafood]",4.5,$$$,"202 State St Santa Barbara, CA 93101",34.414204,-119.691644
4,U3grYFIeu6RgAAQgdriHww,Brophy Bros - Santa Barbara,[Seafood],4.0,$$,"119 Harbor Way Santa Barbara, CA 93109",34.403478,-119.693640
...,...,...,...,...,...,...,...,...
7173,Wy8Hswf2cLQGRZN6armkag,Jack in the Box,"[Fast Food, Burgers, Tacos]",1.5,$,"6875 Hollister Ave Goleta, CA 93117",34.429897,-119.868783
7174,j6TDQvbRoMszGR0hyaVxvw,McDonald's,"[Fast Food, Burgers, Coffee & Tea]",2.5,$,"6900 Marketplace Dr Goleta, CA 93117",34.429026,-119.870127
7175,aD4ysR05o5-onOjYjJ8QiQ,Sam's To Go - Goleta,"[Delis, Sandwiches, Soup]",3.5,$,"6831 Hollister Ave Goleta, CA 93117",34.429902,-119.867168
7176,FjVAwrTr0AMfs78pDLamtA,Subway,[Sandwiches],2.5,$,"6831 Hollister Ave Ste J Goleta, CA 93117",34.429467,-119.867070


In [21]:
# Save the updated dataframe to a new CSV file
df_gaslamp.to_csv('sb_restaurants.csv', index=False)

In [25]:
import requests

# Yelp API endpoint and parameters
business_id = '6EizP0XyqITuGZ8GZMI9Lw'
url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
headers = {
    'Authorization': f'Bearer {api_key_secret}',
    'Content-Type': 'application/json'
}

params = {'limit': 3}

# Send a GET request to the API endpoint with the parameters and headers
response = requests.get(url, headers=headers, params=params)

# Check if there was an error in the request
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
else:
    # Parse the JSON response and extract the relevant information
    data = response.json()
    reviews = []
    for review in data['reviews']:
        reviews.append(review['text'])
    
    # Print the list of reviews
    print(reviews)


["My husband and I spent almost a week in Santa Barbara hitting up all the restaurants we've been wanting to go to. We got married so it was a special...", "10 out of 10 Fantastic!!! \nHad to come out to try Toma since it's been 5 years since my last visit! And I can confidently tell you- I was blown away! \n\nMy...", 'Went here for date night on a Saturday evening with 5:00 pm reservations. We were immediately greeted and seated at our table; our waiter provided us with...']


In [29]:
reviews

["My husband and I spent almost a week in Santa Barbara hitting up all the restaurants we've been wanting to go to. We got married so it was a special...",
 "10 out of 10 Fantastic!!! \nHad to come out to try Toma since it's been 5 years since my last visit! And I can confidently tell you- I was blown away! \n\nMy...",
 'Went here for date night on a Saturday evening with 5:00 pm reservations. We were immediately greeted and seated at our table; our waiter provided us with...']

In [42]:
# Yelp API endpoint and parameters
url_template = 'https://api.yelp.com/v3/businesses/{}/reviews'
headers = {
    'Authorization': f'Bearer {api_key_secret}',
    'Content-Type': 'application/json'
}

params = {'limit': 3}
# Create an empty list to store the review texts for all businesses
reviews_list = []

# Loop over the rows of the dataframe and make separate API requests for each business_id
for idx, row in df_restaurants.iterrows():
    business_id = row['id']
    url = url_template.format(business_id)
    response = requests.get(url, headers=headers, params=params)

    # Check if there was an error in the request
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
    else:
        # Parse the JSON response and extract the relevant information
        data = response.json()
        reviews = []
        for review in data['reviews']:
            reviews.append(review['text'])

       # Append the list of reviews and business ID for this business to the reviews_list
        reviews_list.append([business_id, reviews])

# Create a new dataframe to store the business IDs and reviews
df_reviews = pd.DataFrame(reviews_list, columns=['business_id', 'reviews'])



Error: 403
{"error": {"code": "BUSINESS_UNAVAILABLE", "description": "We may not be able to provide details for certain businesses, for example if they do not have any reviews yet."}}
Error: 403
{"error": {"code": "BUSINESS_UNAVAILABLE", "description": "We may not be able to provide details for certain businesses, for example if they do not have any reviews yet."}}
Error: 403
{"error": {"code": "BUSINESS_UNAVAILABLE", "description": "We may not be able to provide details for certain businesses, for example if they do not have any reviews yet."}}
Error: 403
{"error": {"code": "BUSINESS_UNAVAILABLE", "description": "We may not be able to provide details for certain businesses, for example if they do not have any reviews yet."}}
Error: 403
{"error": {"code": "BUSINESS_UNAVAILABLE", "description": "We may not be able to provide details for certain businesses, for example if they do not have any reviews yet."}}
Error: 403
{"error": {"code": "BUSINESS_UNAVAILABLE", "description": "We may not

In [66]:
df_reviews

,business_id,reviews
0,6EizP0XyqITuGZ8GZMI9Lw,[My husband and I spent almost a week in Santa...
1,tthy48ZIX_wfY8Dy0Dvg6w,"[Seriously, for a special occasion, Bouchon ca..."
2,yPSejq3_erxo9zdVYTBnZA,[Great food! One of the best tacos I ever had....
3,KSYONgGtrK0nKXfroB-bwg,[We had a great experience here for a girls ni...
4,U3grYFIeu6RgAAQgdriHww,[Fresh seafood! Friendly service. Definitely w...
...,...,...
3443,LKM0wL3GMsi8NwEdz-_n8w,[First time coming here driving on the 101 dec...
3444,_nSL2u-ZzX6kkAF2D2EZgQ,[Service- average ***\nBrussels sprouts ****.....
3445,bbEXAEFr4RYHLlZ-HFssTA,"[I stumbled into the original ""The Habit"" afte..."
3446,6kPHr6gZaU9732ptn9gJrA,"[AMAZING HOMEMADE FOOD,Great environment, vege..."


In [76]:
 # Save the reviews dataframe to a CSV file after every restaurant to avoid losing progress
df_reviews.to_csv('reviews_sb.csv', index=False)

In [63]:
df_restaurants = df_restaurants.rename(columns={'id': 'business_id'})


In [78]:
merged_df = df_restaurants.join(df_reviews.set_index('business_id'), on='business_id')



In [85]:
reviews_count = merged_df.groupby('business_id')['reviews'].count()



In [87]:
merged_df_unique = merged_df.drop_duplicates(subset='business_id')


In [89]:
 # Save the reviews dataframe to a CSV file after every restaurant to avoid losing progress
merged_df_unique.to_csv('reviews_sb2.csv', index=False)

In [58]:
 # Save the reviews dataframe to a CSV file after every restaurant to avoid losing progress
reviews_df.to_csv('reviews_df.csv', index=False)

# json file for businesses

In [4]:
import pandas as pd
business_json_path = 'yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

In [12]:
ca_cities_count = df_b[df_b['state'] == 'CA']


In [33]:
#number of different cities
dif_n_cities = df_b['state'].nunique()
print(dif_n_cities)

27


In [34]:
#number of different cities
dif_cities = df_b['state'].unique()
print(dif_cities)

['CA' 'MO' 'AZ' 'PA' 'TN' 'FL' 'IN' 'LA' 'AB' 'NV' 'ID' 'DE' 'IL' 'NJ'
 'NC' 'CO' 'WA' 'HI' 'UT' 'TX' 'MT' 'MI' 'SD' 'XMS' 'MA' 'VI' 'VT']


In [35]:
#number of different states
dif_cities = df_b['city'].nunique()
print(dif_cities)

1416


In [37]:
dif_cities = df_b['city'].unique()
print(dif_cities[:20])

['Santa Barbara' 'Affton' 'Tucson' 'Philadelphia' 'Green Lane'
 'Ashland City' 'Brentwood' 'St. Petersburg' 'Nashville' "Land O' Lakes"
 'Tampa Bay' 'Indianapolis' 'Clearwater' 'Largo' 'New Orleans' 'Kenner'
 'Edmonton' 'Reno' 'Newtown' 'White House']


In [14]:
ca_cities_count.to_csv('ca_cities.csv', index=False)

# json file for reviews

In [39]:
import pandas as pd

# define the file path
review_json_path = 'yelp_academic_dataset_review.json'

# define the chunk size
size = 1000000

# create an empty list to store the chunks
chunks = []

# read the file in chunks and append each chunk to the list
for chunk in pd.read_json(review_json_path, lines=True,
                          dtype={'review_id':str,'user_id':str,
                                 'business_id':str,'stars':int,
                                 'date':str,'text':str,'useful':int,
                                 'funny':int,'cool':int},
                          chunksize=size):
    chunks.append(chunk)

# concatenate the chunks into a single DataFrame
reviews_df = pd.concat(chunks, ignore_index=True)

# display the first few rows of the DataFrame
print(reviews_df.head())



                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm...  2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delici

In [40]:
reviews_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


In [27]:
reviews_df.to_csv('reviews_df_j.csv', index=False)

In [41]:
merged_df = pd.merge(df_b, reviews_df, on='business_id')

In [42]:
merged_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,"Doctors, Traditional Chinese Medicine, Naturop...",None,9vwYDBVI3ymdqcyJ5WW2Tg,e0imecnX_9MtLnS2rUZM-A,5,3,2,1,I've had acupuncture treatments with Abby over...,2012-05-02 18:07:38
1,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,"Doctors, Traditional Chinese Medicine, Naturop...",None,OXgg1LdxHDv3CBU5-xi2lA,_Q0fdLVoTnlNkEypUvNkHA,5,1,0,0,Abby is an amazing practitioner. In a treatmen...,2013-03-01 06:11:05
2,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,"Doctors, Traditional Chinese Medicine, Naturop...",None,DG64cjud9cWB4fANskVxSw,ycUooVIDWPgXPf6niW-FWQ,4,2,0,0,I went to see Abby for some digestive issues. ...,2013-01-17 00:05:43
3,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,"Doctors, Traditional Chinese Medicine, Naturop...",None,hzvRRb40oPttxAdyr7kfow,CiwVvb7jWijWB5jkmatzKA,5,0,1,0,"Abby helped me with some longstanding issues, ...",2015-03-16 03:43:08
4,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,"Doctors, Traditional Chinese Medicine, Naturop...",None,xUkBPk-QfcW4i3MRU5TeXw,QkCbMKBktkrkOFJugHvY6w,5,0,0,0,"Recently, I referred a patient of mine with mu...",2013-03-05 18:45:07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990275,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,...,"Beauty & Spas, Permanent Makeup, Piercing, Tattoo","{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...",b4lUxYu-8yBkq8H4NgzS2A,EYKtdSmIb48URxUagC937A,5,0,0,0,It is with my greatest pleasure to say that I'...,2017-10-31 02:29:12
6990276,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,...,"Beauty & Spas, Permanent Makeup, Piercing, Tattoo","{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...",5gE7ACSqGdQSaiS75Nl_tw,T-tMouhG-faaEyh2zpprYg,5,0,0,0,I went there to get my nose piercing replaced....,2021-07-29 21:11:02
6990277,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,...,"Beauty & Spas, Permanent Makeup, Piercing, Tattoo","{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...",34FBVC0N2M-7r2aCZnXecA,r_xuBblpy45w9lPHUwQFtA,5,0,0,0,Fantastic tattoo artist. Friendly and talkativ...,2019-08-26 05:44:13
6990278,jV_XOycEzSlTx-65W906pg,Sic Ink,238 Apollo Beach Blvd,Apollo beach,FL,33572,27.771002,-82.394910,4.5,9,...,"Beauty & Spas, Permanent Makeup, Piercing, Tattoo","{'Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...",VSMXMQ8qOauSGS4sC_4XHQ,wZw_sEDCvTxV9Zc4ftHb5w,5,1,0,1,"Please note, this studio has relocated to Apol...",2020-02-10 00:30:15


In [45]:
merged_df = merged_df[merged_df['state'] == 'CA']

In [48]:
merged_df.to_csv('CA_business_reviews.csv', index=False)

In [57]:
import requests
import json

zipcode = '93105.0'

def get_businesses(api_key, zipcode):
    headers = {
        'Authorization': 'Bearer ' + api_key,
    }
    
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        'location': zipcode,
        'categories': 'restaurants',
        'limit': 1,
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

def get_reviews(api_key, business_id):
    headers = {
        'Authorization': 'Bearer ' + api_key,
    }
    
    url = f"https://api.yelp.com/v3/businesses/{business_id}/reviews"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

businesses_data = get_businesses(api_key_secret, zipcode)

if businesses_data is not None and businesses_data["businesses"]:
    business_id = businesses_data["businesses"][0]["id"]
    reviews_data = get_reviews(api_key_secret, business_id)

    if reviews_data is not None:
        print(f"Reviews for business ID {business_id}:")
        for review in reviews_data["reviews"]:
            print(f"{review['user']['name']}: {review['text']}")
    else:
        print(f"Failed to fetch reviews for business ID {business_id}")
else:
    print(f"No businesses found in zip code {zipcode}")

No businesses found in zip code 93105.0
